# Data Management

In [1]:
import pandas as pd
import numpy as np
import re

The original dataframe "df" contains the date, distance, and runtime of each run from 2010-present. The goal of this data management process is to calculate a "pace" column from the distance and time of each run.

In [2]:
df = pd.read_excel("Running_Log.xlsx")

In [3]:
pd.options.display.max_rows=20
print(df)

            Date  Mileage     Time
0     2009-01-05     1.00      NaN
1     2009-04-25     3.10    29:38
2     2009-11-21     3.10    22:51
3     2010-01-04     1.30    17:00
4     2010-01-05     2.60    28:00
5     2010-01-07     2.60    28:00
6     2010-01-11     1.30      NaN
7     2010-01-14     2.60    26:00
8     2010-01-16     4.05    35:48
9     2010-01-19     2.60    24:00
...          ...      ...      ...
3013  2022-11-16     5.01    38:57
3014  2022-11-18     6.29    43:58
3015  2022-11-19     3.11    17:08
3016  2022-11-19     4.33      NaN
3017  2022-11-21     6.50    50:18
3018  2022-11-23     6.91  1:01:59
3019  2022-11-24     3.32    17:58
3020  2022-11-24     4.81      NaN
3021  2022-11-26     6.22    40:49
3022  2022-11-29     5.00    36:23

[3023 rows x 3 columns]


The main obstacle in creating a pace column is that it is not easy to do math with times in mm:ss format. So the "Time" column must be reformatted into minutes, with fractions of minutes put in decimal format (e.g. instead of "29:38", output = "29.633").

In [4]:
print(df.iloc[[0,1389]])

            Date  Mileage   Time
0     2009-01-05     1.00    NaN
1389  2016-11-26     6.13  46:34


However, as can be seen above, some of my runs lasted longer than an hour and were thus input in h:mm:ss format. When I attempted to delimit the col by ":", the "1" (hour) in row 1389 was put in the same column as the "29" (minutes) in row 0.

To work around this problem, I computed a new col "time_h" with a standardized time format of "h:mm:ss" as seen below. I added an assert() statement to ensure that all data points in "time_h" started with a number and then a colon, as I had intended.

In [5]:
# added new col to standardize format of time var as "h:mm:ss"
df["time_h"] = pd.np.where(df.Time.str.contains('\d:\d\d:\d\d', regex = True), df["Time"], "0:" + df["Time"])
print(df.iloc[[0,1389]])

testseries = df['time_h'].dropna()
for i in testseries:
    assert(re.match('\d:*', i))

            Date  Mileage   Time   time_h
0     2009-01-05     1.00    NaN      NaN
1389  2016-11-26     6.13  46:34  0:46:34


Then I created a new dataframe where df['time_h'] was delimited by ":" into three columns.

Next, I merged the "new" dataframe with the "df" dataframe and called the resulting dataframe "result".

Finally, I created "df2" from "result" and renamed the vars from "new" to be "hours", "minutes", and "seconds".

These changes can be seen in the following three sections of code.

In [6]:
# created new dataframe with "time_h" col expanded into hr, min, and sec vars
new = df["time_h"].str.split(pat = ":", expand = True)
print(new)

        0    1    2
0     NaN  NaN  NaN
1       0   29   38
2       0   22   51
3       0   17   00
4       0   28   00
5       0   28   00
6     NaN  NaN  NaN
7       0   26   00
8       0   35   48
9       0   24   00
...   ...  ...  ...
3013    0   38   57
3014    0   43   58
3015    0   17   08
3016  NaN  NaN  NaN
3017    0   50   18
3018    1   01   59
3019    0   17   58
3020  NaN  NaN  NaN
3021    0   40   49
3022    0   36   23

[3023 rows x 3 columns]


In [7]:
result = pd.concat([df, new], axis=1, sort=False)
print(result)

            Date  Mileage     Time   time_h    0    1    2
0     2009-01-05     1.00      NaN      NaN  NaN  NaN  NaN
1     2009-04-25     3.10    29:38  0:29:38    0   29   38
2     2009-11-21     3.10    22:51  0:22:51    0   22   51
3     2010-01-04     1.30    17:00  0:17:00    0   17   00
4     2010-01-05     2.60    28:00  0:28:00    0   28   00
5     2010-01-07     2.60    28:00  0:28:00    0   28   00
6     2010-01-11     1.30      NaN      NaN  NaN  NaN  NaN
7     2010-01-14     2.60    26:00  0:26:00    0   26   00
8     2010-01-16     4.05    35:48  0:35:48    0   35   48
9     2010-01-19     2.60    24:00  0:24:00    0   24   00
...          ...      ...      ...      ...  ...  ...  ...
3013  2022-11-16     5.01    38:57  0:38:57    0   38   57
3014  2022-11-18     6.29    43:58  0:43:58    0   43   58
3015  2022-11-19     3.11    17:08  0:17:08    0   17   08
3016  2022-11-19     4.33      NaN      NaN  NaN  NaN  NaN
3017  2022-11-21     6.50    50:18  0:50:18    0   50   

In [8]:
df2 = result.rename(columns={"time_h": "time_corr", 0: "hours", 1: "minutes", 2: "seconds"})
print(df2)

            Date  Mileage     Time time_corr hours minutes seconds
0     2009-01-05     1.00      NaN       NaN   NaN     NaN     NaN
1     2009-04-25     3.10    29:38   0:29:38     0      29      38
2     2009-11-21     3.10    22:51   0:22:51     0      22      51
3     2010-01-04     1.30    17:00   0:17:00     0      17      00
4     2010-01-05     2.60    28:00   0:28:00     0      28      00
5     2010-01-07     2.60    28:00   0:28:00     0      28      00
6     2010-01-11     1.30      NaN       NaN   NaN     NaN     NaN
7     2010-01-14     2.60    26:00   0:26:00     0      26      00
8     2010-01-16     4.05    35:48   0:35:48     0      35      48
9     2010-01-19     2.60    24:00   0:24:00     0      24      00
...          ...      ...      ...       ...   ...     ...     ...
3013  2022-11-16     5.01    38:57   0:38:57     0      38      57
3014  2022-11-18     6.29    43:58   0:43:58     0      43      58
3015  2022-11-19     3.11    17:08   0:17:08     0      17    

To calculate pace, I needed to get my three time columns into numeric format.

Then I needed to combine them back into one time variable in seconds.

In [9]:
df2['hours'] = pd.to_numeric(df2['hours'])
df2['minutes'] = pd.to_numeric(df2['minutes'])
df2['seconds'] = pd.to_numeric(df2['seconds'])

In [10]:
time_seconds = df2.hours*3600 + df2.minutes*60 + df2.seconds
df2['time_s'] = time_seconds
print(df2)

            Date  Mileage     Time time_corr  hours  minutes  seconds  time_s
0     2009-01-05     1.00      NaN       NaN    NaN      NaN      NaN     NaN
1     2009-04-25     3.10    29:38   0:29:38    0.0     29.0     38.0  1778.0
2     2009-11-21     3.10    22:51   0:22:51    0.0     22.0     51.0  1371.0
3     2010-01-04     1.30    17:00   0:17:00    0.0     17.0      0.0  1020.0
4     2010-01-05     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0
5     2010-01-07     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0
6     2010-01-11     1.30      NaN       NaN    NaN      NaN      NaN     NaN
7     2010-01-14     2.60    26:00   0:26:00    0.0     26.0      0.0  1560.0
8     2010-01-16     4.05    35:48   0:35:48    0.0     35.0     48.0  2148.0
9     2010-01-19     2.60    24:00   0:24:00    0.0     24.0      0.0  1440.0
...          ...      ...      ...       ...    ...      ...      ...     ...
3013  2022-11-16     5.01    38:57   0:38:57    0.0     38.0    

At last, I was able to calculate pace (after converting the column for distance to numeric format). I first calculated it in seconds-per-mile.

Then I created a new column in which the units for pace were a more meaningful minutes-per-mile.

In [11]:
df2['Mileage'] = pd.to_numeric(df2['Mileage'])
pace_s = df2.time_s / df2.Mileage
df2['pace_s'] = pace_s
print(df2)

            Date  Mileage     Time time_corr  hours  minutes  seconds  time_s  \
0     2009-01-05     1.00      NaN       NaN    NaN      NaN      NaN     NaN   
1     2009-04-25     3.10    29:38   0:29:38    0.0     29.0     38.0  1778.0   
2     2009-11-21     3.10    22:51   0:22:51    0.0     22.0     51.0  1371.0   
3     2010-01-04     1.30    17:00   0:17:00    0.0     17.0      0.0  1020.0   
4     2010-01-05     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0   
5     2010-01-07     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0   
6     2010-01-11     1.30      NaN       NaN    NaN      NaN      NaN     NaN   
7     2010-01-14     2.60    26:00   0:26:00    0.0     26.0      0.0  1560.0   
8     2010-01-16     4.05    35:48   0:35:48    0.0     35.0     48.0  2148.0   
9     2010-01-19     2.60    24:00   0:24:00    0.0     24.0      0.0  1440.0   
...          ...      ...      ...       ...    ...      ...      ...     ...   
3013  2022-11-16     5.01   

In [12]:
pace_min = df2.pace_s / 60
df2['pace_min'] = pace_min
print(df2)

            Date  Mileage     Time time_corr  hours  minutes  seconds  time_s  \
0     2009-01-05     1.00      NaN       NaN    NaN      NaN      NaN     NaN   
1     2009-04-25     3.10    29:38   0:29:38    0.0     29.0     38.0  1778.0   
2     2009-11-21     3.10    22:51   0:22:51    0.0     22.0     51.0  1371.0   
3     2010-01-04     1.30    17:00   0:17:00    0.0     17.0      0.0  1020.0   
4     2010-01-05     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0   
5     2010-01-07     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0   
6     2010-01-11     1.30      NaN       NaN    NaN      NaN      NaN     NaN   
7     2010-01-14     2.60    26:00   0:26:00    0.0     26.0      0.0  1560.0   
8     2010-01-16     4.05    35:48   0:35:48    0.0     35.0     48.0  2148.0   
9     2010-01-19     2.60    24:00   0:24:00    0.0     24.0      0.0  1440.0   
...          ...      ...      ...       ...    ...      ...      ...     ...   
3013  2022-11-16     5.01   

I noticed that, due to some input errors in the original excel file, the formatting for all values in the "Date" column between the years 2012 and 2014 included the time of day (which, as I had never input that, ended up as a bunch of zeros at the end of the cell, as can be seen below in row 800).

I fixed this by changing the Date column to string format and then keeping only the first ten characters in each cell of that column.

In [13]:
print(df2.iloc[[0,800]])

                    Date  Mileage   Time time_corr  hours  minutes  seconds  \
0             2009-01-05      1.0    NaN       NaN    NaN      NaN      NaN   
800  2014-12-15 00:00:00      6.3  46:22   0:46:22    0.0     46.0     22.0   

     time_s      pace_s  pace_min  
0       NaN         NaN       NaN  
800  2782.0  441.587302  7.359788  


In [14]:
df2.Date = df2.Date.astype(str)
df2['Date'] = df2['Date'].str[:10]
print(df2)

            Date  Mileage     Time time_corr  hours  minutes  seconds  time_s  \
0     2009-01-05     1.00      NaN       NaN    NaN      NaN      NaN     NaN   
1     2009-04-25     3.10    29:38   0:29:38    0.0     29.0     38.0  1778.0   
2     2009-11-21     3.10    22:51   0:22:51    0.0     22.0     51.0  1371.0   
3     2010-01-04     1.30    17:00   0:17:00    0.0     17.0      0.0  1020.0   
4     2010-01-05     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0   
5     2010-01-07     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0   
6     2010-01-11     1.30      NaN       NaN    NaN      NaN      NaN     NaN   
7     2010-01-14     2.60    26:00   0:26:00    0.0     26.0      0.0  1560.0   
8     2010-01-16     4.05    35:48   0:35:48    0.0     35.0     48.0  2148.0   
9     2010-01-19     2.60    24:00   0:24:00    0.0     24.0      0.0  1440.0   
...          ...      ...      ...       ...    ...      ...      ...     ...   
3013  2022-11-16     5.01   

As a final step before completing the data management process, I created a new dataframe df3 which subset df2 into only the columns I needed. Then I wanted to view the entire dataset to ensure that there were no hidden mistakes.

In [15]:
df3 = df2[['Date','Mileage','time_corr','pace_min']]
pd.options.display.max_rows=3000
#print(df3)
print(df3.tail(10))
pd.options.display.max_rows=20

            Date  Mileage time_corr  pace_min
3013  2022-11-16     5.01   0:38:57  7.774451
3014  2022-11-18     6.29   0:43:58  6.989931
3015  2022-11-19     3.11   0:17:08  5.509110
3016  2022-11-19     4.33       NaN       NaN
3017  2022-11-21     6.50   0:50:18  7.738462
3018  2022-11-23     6.91   1:01:59  8.970092
3019  2022-11-24     3.32   0:17:58  5.411647
3020  2022-11-24     4.81       NaN       NaN
3021  2022-11-26     6.22   0:40:49  6.562165
3022  2022-11-29     5.00   0:36:23  7.276667


Finally, I exported df3 to a csv so that I have externally saved changes. This csv will be read in at the beginning of the df_visualization and df_analysis ipynb's.

In [16]:
df3.to_csv("clean.csv", index=False)